In [1]:
import os

Libraries/Packages

In [2]:
#general
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt

### Preparing explanatory variables

In [3]:
df =  pd.read_parquet("/Users/carrot2506/Files on Lap/Thesis Preparation/Data/cleaned_TAQ_IBM_N_2020_Feb_Mar_Apr_Thu.parquet", engine="pyarrow")

In [4]:
print(df.head())

       Date     TradeTime   Price  Volume     QuoteTime    Bid     Ask  \
0  20200203  34241.145687  144.30      22  34241.145661  144.2  144.38   
1  20200203  34241.145696  144.27       5  34241.145661  144.2  144.38   
2  20200203  34241.145748  144.25      73  34241.145661  144.2  144.38   
3  20200203  34241.153160  144.35       4  34241.148387  144.2  144.37   
4  20200203  34241.153276  144.30      10  34241.148387  144.2  144.37   

   BidAskSpread  
0          0.18  
1          0.18  
2          0.18  
3          0.17  
4          0.17  


#### Time between trades

In [5]:
df['delta_T'] = df.groupby('Date')['TradeTime'].shift(1).rsub(df['TradeTime'])

In [ ]:
#max_deltas = df.groupby('Date')['delta_T'].max()
#print(max_deltas.sort_values(ascending=False).head(50))

In [7]:
print(df['delta_T'].describe())

count    876761.000000
mean          6.663154
std           6.548076
min           1.000000
25%           3.000000
50%           5.000000
75%           8.000000
max         251.000000
Name: delta_T, dtype: float64


In [6]:
df['delta_T_scaled100'] = df['delta_T'] / 100

In [7]:
df['delta_T_scaled10'] = df['delta_T'] / 10

In [10]:
print(df['delta_T_scaled10'].describe())

count    876761.000000
mean          0.666315
std           0.654808
min           0.100000
25%           0.300000
50%           0.500000
75%           0.800000
max          25.100000
Name: delta_T_scaled10, dtype: float64


#### First-lag of Bid/Ask Spread

In [8]:
df['AB_lag1'] = df['BidAskSpread'].shift(1)

#### Volume

In [9]:
percentiles = [0, 25, 50, 75, 90, 99, 99.5, 100]
volume_percentiles = np.percentile(df['Volume'], percentiles).astype(int)


percentile_df = pd.DataFrame({
    'Percentile (%)': percentiles,
    'Volume': volume_percentiles
})

print("Volume Distribution:")
print(percentile_df)

Volume Distribution:
   Percentile (%)  Volume
0             0.0       1
1            25.0      13
2            50.0      85
3            75.0     100
4            90.0     158
5            99.0     470
6            99.5     651
7           100.0   55653


In [10]:
df['V'] = df['Volume'].apply(lambda x: min(x, 651)) #Hausman et al (1992): if the trade exceeds 99.5 percentile -> set it equal to 99.5 percentile

df['V'] = df['Price'] * df['V'] #calculate dollar volume
df['V'] = df['V']/100 #denominate in $100 of dollar

In [11]:
df['lnV'] = np.log(df['V'])

In [ ]:
#V_percentiles = np.percentile(df['V'], percentiles).astype(int)


#dollar_V_percentile_df = pd.DataFrame({
 #   'Percentile (%)': percentiles,
  #  'Dollar Volume': V_percentiles
#})

#print("Dollar Volume Distribution:")
#print(dollar_V_percentile_df)

Dollar Volume Distribution:
   Percentile (%)  Dollar Volume
0             0.0              1
1            25.0             14
2            50.0             73
3            75.0            142
4            99.5            929
5           100.0           1210


#### Buyer/Seller-initiated Indicator

In [12]:
df['Z'] = df.groupby('Date')['Price'].shift(1).rsub(df['Price']) #Price change

In [13]:
df['MidQuote'] = (df['Bid'] + df['Ask']) / 2 #Midquote

In [14]:
#If trade price > midquote: buyer initiated: IBS=1
#If trade price < midquote: seller initiated: IBS=−1
#If trade price = midquote: “Tick test”, i.e.:
#If midquote > previous trade price: buyer initiated
#If midquote < previous trade price: seller initiated
#If there is no price change between consecutive trades: classification based on last tick change (if the previous tick change is up then the trade is defined as buyer initiated and vice versa.)


def last_tick_change(idx, tick_changes):
    valid_indices = tick_changes.index
    for i in range(valid_indices.get_loc(idx) - 1, -1, -1):
        if tick_changes.iloc[i] != 0:
            return tick_changes.iloc[i]
    return 0

df['IBS'] = 0
df['PrevPrice'] = df['Price'].shift(1)

groups = df.groupby(df['Date']) #By date!!

for date, daily_df in groups:

    daily_df.loc[daily_df['Price'] > daily_df['MidQuote'], 'IBS'] = 1
    daily_df.loc[daily_df['Price'] < daily_df['MidQuote'], 'IBS'] = -1

    tick_test = daily_df['Price'] == daily_df['MidQuote']

    daily_df.loc[tick_test & (daily_df['MidQuote'] > daily_df['PrevPrice']), 'IBS'] = 1
    daily_df.loc[tick_test & (daily_df['MidQuote'] < daily_df['PrevPrice']), 'IBS'] = -1

    no_change_idxs = daily_df[tick_test & (daily_df['MidQuote'] == daily_df['PrevPrice'])].index
    for idx in no_change_idxs:
        if idx in daily_df.index:
            last_tick = last_tick_change(idx, daily_df['Z'])
            if last_tick > 0:
                daily_df.loc[idx, 'IBS'] = 1
            elif last_tick < 0:
                daily_df.loc[idx, 'IBS'] = -1

    df.loc[daily_df.index, 'IBS'] = daily_df['IBS']

#### Save output

In [15]:
output_file = os.path.join(os.getcwd(), "prep_IBM_N_2020_Feb_Mar_Apr_Thu.parquet") #backup
df.to_parquet(output_file)